In [ ]:
import sys
sys.path.append("..")
from src.model.components.clip_text_autoencoder import CLIPAutoEncoder
from src.dataset.text import SimpleTokenizer
from pathlib import Path
import random
from src.utils.datatool import read_jsonlines
import torch

from lmdb_embeddings.reader import LmdbEmbeddingsReader

In [ ]:
descriptions = set()
samples = read_jsonlines("/data/vtt/meta/vtt_all.jsonl")
for sample in samples:
    for label in sample["annotation"]:
        descriptions.add(label["label"])
descriptions = list(sorted(list(descriptions)))
print(f"Total descriptions: {len(descriptions)}")
print(descriptions[:10])

In [ ]:
# vitb16_4
ckpt_path_list = {
    # "vitb16_4_50e": Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-14_00-11-16/checkpoints/last.ckpt"),
    # "vitb16_4_300e": Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-20_12-35-24/checkpoints/last.ckpt"),
    "vitb16_4": Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-20_12-35-27/checkpoints/last.ckpt"),
    "vitb32_4": Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-20_12-35-26/checkpoints/last.ckpt"),
    "vitl14_4": Path("/log/exp/vtt/WikiHowDataModule.CLIPAutoEncoder.GenerationLoss.2023-01-20_12-35-28/checkpoints/last.ckpt")
}
models = {}
for name, ckpt_path in ckpt_path_list.items():
    models[name] = CLIPAutoEncoder(from_decoder_ckpt=ckpt_path)
    models[name].eval()
    models[name].cuda()
tokenizer = SimpleTokenizer()

In [ ]:
# random.seed(0)
for i in range(5):
    text = random.choice(descriptions)
    print(f"Text: {text}")
    print(f"De-text: ")
    for name, model in models.items():
        embedding = model.encode_raw(text)
        de_text = model.decode_raw(embedding)
        print(f"  {name}: {de_text}")
    print()
